# **自定義訓練流程（Custom training）**
此份程式碼會介紹如何建立自定義的 dataset, model, losses 以及透過 tf.GradinetType 去訓練模型。

## 本章節內容大綱
* ### [建立資料集](#CreateDataset)
* ### [建構模型](#BuildModel)
* ### [訓練模型](#TrainModel)
* ### [評估模型](#EvaluateModel)
---

## 匯入套件

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Tensorflow 相關套件
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

<a name="CreateDataset"></a>
## 建立資料集

In [ ]:
# 上傳資料
!wget -q https://github.com/TA-aiacademy/course_3.0/releases/download/DL/Data_part4.zip
!unzip -q Data_part4.zip

In [ ]:
df = pd.read_csv('./Data/bodyperformance.csv')
df.head()

* #### 身體素質資料集
共 13393 筆，11 種身體體能表現相關特徵，類別共 4 種，0 等為最優依序排列至 3 等。

In [ ]:
X = df.iloc[:, :-1].values
y = df['class'].values

In [ ]:
y_onehot = keras.utils.to_categorical(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y_onehot,
                                                      test_size=0.2,
                                                      random_state=17,
                                                      stratify=y)

In [ ]:
print(f'X_train shape: {X_train.shape}')
print(f'X_valid shape: {X_valid.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_valid shape: {y_valid.shape}')

In [ ]:
# Feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train, y_train)
X_valid = sc.transform(X_valid)

In [ ]:
batch_size = 64

# 準備訓練資料集
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(
    buffer_size=1024,
    seed=17).batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()

# 準備驗證資料集
val_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
val_dataset = val_dataset.batch(batch_size)

<a name="BuildModel"></a>
## 建構模型

In [ ]:
class my_net(keras.Model):  # build model object by custom class
    def __init__(self, num_classes=4):
        super(my_net, self).__init__()
        keras.backend.clear_session()  # 重置 keras 的所有狀態
        tf.random.set_seed(17)  # 設定 tensorflow 隨機種子
        self.input_layer = layers.Input(shape=(11,))
        self.hidden_layer_1 = layers.Dense(
            32,  # 神經元個數
            activation='swish')
        self.hidden_layer_2 = layers.Dense(
            32,  # 神經元個數
            activation='swish')
        self.output_layer = layers.Dense(
            num_classes,
            activation='softmax')
        self.out = self.call(self.input_layer)

    def call(self, inputs):
        x = self.hidden_layer_1(inputs)
        x = self.hidden_layer_2(x)
        outputs = self.output_layer(x)
        return outputs

In [ ]:
model = my_net()
model.build(input_shape=(None, 11))
model.summary()

In [ ]:
class my_crossentropy(keras.losses.Loss):  # build loss object by custom class
    def call(self, y_true, y_pred):
        return keras.losses.categorical_crossentropy(y_true,
                                                     y_pred,
                                                     from_logits=False)

In [ ]:
# 創建損失函數
loss_fn = my_crossentropy()
# 創建優化器
optimizer = keras.optimizers.Nadam()

# 創建評估函數
train_acc_metric = keras.metrics.CategoricalAccuracy()
val_acc_metric = keras.metrics.CategoricalAccuracy()

<a name="TrainModel"></a>
## 訓練模型

In [ ]:
import time
import tqdm

# 創建 list 分別存放訓練集 acc, loss 和驗證集 acc
train_acc_list, train_loss_list = [], []
val_acc_list, val_loss_list = [], []

epochs = 10

# 訓練的迭代過程
for epoch in range(epochs):
    start_time = time.time()
    t_bar = tqdm.tqdm_notebook(enumerate(train_dataset),
                               total=len(train_dataset),
                               desc=f'Epoch {epoch}')

    # 每次的迭代讀取一個批次的資料量
    for step, (x_batch_train, y_batch_train) in t_bar:
        with tf.GradientTape() as tape:
            outputs = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, outputs)

        grads = tape.gradient(loss_value, model.trainable_weights)  # 計算參數上的梯度
        optimizer.apply_gradients(zip(grads, model.trainable_weights))  # 更新參數

        train_acc_metric.update_state(y_batch_train, outputs)  # 存放每個批次的評估結果

    # 印出每個迭代回合的訓練評估結果
    print('Training loss over epoch: %.4f' % (float(loss_value),))
    train_acc = train_acc_metric.result()  # 平均所有存放的評估結果
    print('Training acc over epoch: %.4f' % (float(train_acc),))

    # 將訓練的評估結果儲存下來
    train_acc_list.append(train_acc)
    train_loss_list.append(loss_value)

    train_acc_metric.reset_states()  # 重置訓練集的評估函數

    # 驗證集的迭代結果
    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        val_acc_metric.update_state(y_batch_val, val_logits)  # 存放每個批次的評估結果

    val_loss = loss_fn(y_batch_val, val_logits)  # 計算最後批次的損失值

    # 印出每個迭代回合的驗證評估結果
    print('Validation loss: %.4f' % (float(val_loss),))
    val_acc = val_acc_metric.result()  # 平均所有存放的評估結果
    print('Validation acc: %.4f' % (float(val_acc),))

    # 將驗證的評估結果儲存下來
    val_acc_list.append(val_acc)
    val_loss_list.append(val_loss)

    val_acc_metric.reset_states()  # 重置驗證集的評估函數

    print('Time taken: %.2fs' % (time.time() - start_time))

<a name="EvaluateModel"></a>
## 評估模型

* ### 視覺化訓練過程的評估指標 （Visualization）

In [ ]:
plt.figure(figsize=(15, 4))
plt.subplot(1, 2, 1)
plt.plot(range(len(train_loss_list)), train_loss_list, label='train_loss')
plt.plot(range(len(val_loss_list)), val_loss_list, label='valid_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(len(train_acc_list)), train_acc_list, label='train_acc')
plt.plot(range(len(val_acc_list)), val_acc_list, label='valid_acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

* ### 模型預測（Model predictions）

In [ ]:
val_pred = []
for x_val, y_val in val_dataset:
    val_pred += list(model.predict(x_val).argmax(-1).flatten())

In [ ]:
val_pred[:10]

In [ ]:
len(val_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_valid.argmax(-1), val_pred))

## 使用 tf.function 加快訓練速度

In [ ]:
model = my_net()
model.build(input_shape=(None, 11))
model.summary()

In [ ]:
# 創建損失函數
loss_fn = my_crossentropy()
# 創建優化器
optimizer = keras.optimizers.Nadam()

# 創建評估函數
train_acc_metric = keras.metrics.CategoricalAccuracy()
val_acc_metric = keras.metrics.CategoricalAccuracy()

In [ ]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        outputs = model(x, training=True)
        loss_value = loss_fn(y, outputs)
    grads = tape.gradient(loss_value, model.trainable_weights)  # 計算參數上的梯度
    optimizer.apply_gradients(zip(grads, model.trainable_weights))  # 更新參數
    train_acc_metric.update_state(y, outputs)  # 存放評估結果
    return loss_value

In [ ]:
@tf.function
def test_step(x, y):
    val_outputs = model(x, training=False)
    val_acc_metric.update_state(y, val_outputs)  # 存放評估結果

In [ ]:
import time
import tqdm

# 創建 list 分別存放訓練集 acc, loss 和驗證集 acc
train_acc_list, train_loss_list = [], []
val_acc_list, val_loss_list = [], []

epochs = 10

# 訓練的迭代過程
for epoch in range(epochs):
    start_time = time.time()
    t_bar = tqdm.tqdm_notebook(enumerate(train_dataset),
                               total=len(train_dataset),
                               desc=f'Epoch {epoch}')

    # 每次的迭代讀取一個批次的資料量
    for step, (x_batch_train, y_batch_train) in t_bar:
        loss_value = train_step(x_batch_train, y_batch_train)

    # 印出每個迭代回合的訓練評估結果
    print("Training loss over epoch: %.4f" % (float(loss_value),))
    train_acc = train_acc_metric.result()  # 平均所有存放的評估結果
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # 將訓練的評估結果儲存下來
    train_acc_list.append(train_acc)
    train_loss_list.append(loss_value)

    train_acc_metric.reset_states()  # 重置訓練集的評估函數

    # 驗證集的迭代結果
    for x_batch_val, y_batch_val in val_dataset:
        test_step(x_batch_val, y_batch_val)

    # 計算最後批次的損失值
    val_logits = model(x_batch_val, training=False)
    val_loss = loss_fn(y_batch_val, val_logits)

    # 印出每個迭代回合的驗證評估結果
    print('Validation loss: %.4f' % (float(val_loss),))
    val_acc = val_acc_metric.result()  # 平均所有存放的評估結果
    print('Validation acc: %.4f' % (float(val_acc),))

    # 將驗證的評估結果儲存下來
    val_acc_list.append(val_acc)
    val_loss_list.append(val_loss)

    val_acc_metric.reset_states()  # 重置驗證集的評估函數

    print('Time taken: %.2fs\n' % (time.time() - start_time))

In [ ]:
# 繪製訓練過程中的評估指標
plt.figure(figsize=(15, 4))
plt.subplot(1, 2, 1)
plt.plot(range(len(train_loss_list)), train_loss_list, label='train_loss')
plt.plot(range(len(val_loss_list)), val_loss_list, label='valid_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(len(train_acc_list)), train_acc_list, label='train_acc')
plt.plot(range(len(val_acc_list)), val_acc_list, label='valid_acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()